(https://github.com/spacetelescope/notebooks/blob/master/notebooks/MAST/PanSTARRS/PS1_DR2_TAP/PS1_DR2_TAP.ipynb)

In [1]:
# Use the pyvo library as our client to the data service.
import pyvo as vo

# For resolving objects with tools from MAST
from astroquery.mast import Mast

# For handling ordinary astropy Tables in responses
from astropy.table import Table

# For displaying and manipulating some types of results
%matplotlib inline
import requests
import astropy
import numpy as np
import pylab
import time
import json
from matplotlib import pyplot as plt

# For calling the object name resolver. Note these are Python 3 dependencies
#import sys
#import http.client as httplib 
#from urllib.parse import quote as urlencode
#from urllib.request import urlretrieve

# To allow display tweaks for wider response tables
from IPython.core.display import display
from IPython.core.display import HTML

# suppress unimportant unit warnings from many TAP services
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.elements")

In [50]:
TAP_service = vo.dal.TAPService("https://vao.stsci.edu/PS1DR2/tapservice.aspx")
TAP_service.describe()

Capability ivo://ivoa.net/std/TAP

Interface None
    http://vao.stsci.edu/ps1dr2/tapservice.aspx

Language ADQL
    ivo://ivoa.net/std/TAPRegExt#features-adqlgeo
        POINT

        CIRCLE

        CONTAINS


Output format application/x-votable+xml
    Also available as votable

Output format text/csv;header=present
    Also available as csv


Capability ivo://ivoa.net/std/DALI#examples-1.0

Interface vr:WebBrowser
    http://vao.stsci.edu/ps1dr2/tapservice.aspx/examples




In [3]:
TAP_tables = TAP_service.tables
for tablename in TAP_tables.keys():
    if not "tap_schema" in tablename:  
        TAP_tables[tablename].describe()
        print("Columns={}".format(sorted([k.name for k in TAP_tables[tablename].columns ])))
        print("----")

dbo.ForcedMeanLensing
    Contains the mean Kaiser et al. (1995) lensing parameters measured from the forced photometry of objects detected in stacked images on the individual single epoch data.  References: Kaiser, N., Squires, G., and Broadhurst, T. 1995, ApJ, 449, 460.

Columns=['batchID', 'gLensObjShearE1', 'gLensObjShearE2', 'gLensObjShearX11', 'gLensObjShearX12', 'gLensObjShearX22', 'gLensObjSmearE1', 'gLensObjSmearE2', 'gLensObjSmearX11', 'gLensObjSmearX12', 'gLensObjSmearX22', 'gLensPSFShearE1', 'gLensPSFShearE2', 'gLensPSFShearX11', 'gLensPSFShearX12', 'gLensPSFShearX22', 'gLensPSFSmearE1', 'gLensPSFSmearE2', 'gLensPSFSmearX11', 'gLensPSFSmearX12', 'gLensPSFSmearX22', 'iLensObjShearE1', 'iLensObjShearE2', 'iLensObjShearX11', 'iLensObjShearX12', 'iLensObjShearX22', 'iLensObjSmearE1', 'iLensObjSmearE2', 'iLensObjSmearX11', 'iLensObjSmearX12', 'iLensObjSmearX22', 'iLensPSFShearE1', 'iLensPSFShearE2', 'iLensPSFShearX11', 'iLensPSFShearX12', 'iLensPSFShearX22', 'iLensPSFSmearE1', '

In [4]:
job = TAP_service.run_async("""
SELECT objID, RAMean, DecMean, nDetections, ng, nr, ni, nz, ny, gMeanPSFMag, rMeanPSFMag, iMeanPSFMag, zMeanPSFMag, yMeanPSFMag
FROM dbo.MeanObjectView
WHERE
CONTAINS(POINT('ICRS', RAMean, DecMean),CIRCLE('ICRS',187.706,12.391,.2))=1
AND nDetections > 1
  """)
TAP_results = job.to_table()
TAP_results

objID,RAMean,DecMean,nDetections,ng,nr,ni,nz,ny,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
,deg,deg,,,,,,,mag,mag,mag,mag,mag
int64,float64,float64,int16,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32
122861877059169881,187.705910033096,12.391133459974,54,10,10,14,8,12,10.295,10.732,11.076,11.342,11.557
122861877056308967,187.70559948,12.39030211,2,0,0,0,0,2,-999.0,-999.0,-999.0,-999.0,14.9985
122871877063310741,187.706311187321,12.3918107963086,3,0,2,0,1,0,-999.0,14.4229,-999.0,14.4249,-999.0
122861877058698594,187.70580919,12.39013311,2,0,0,0,0,2,-999.0,-999.0,-999.0,-999.0,14.7305
122861877050678994,187.705125426086,12.3904305570599,5,2,2,1,0,0,17.0287,15.0843,16.169,-999.0,-999.0
122871877062551436,187.70622365,12.39236505,2,0,0,0,0,2,-999.0,-999.0,-999.0,-999.0,15.1102
122861877044629638,187.70451598,12.39086724,2,0,0,0,2,0,-999.0,-999.0,-999.0,15.5294,-999.0
122861877056688054,187.70558996,12.38960002,2,0,0,0,2,0,-999.0,-999.0,-999.0,14.4064,-999.0


In [6]:
TAP_service.upload_methods

[]

In [9]:
job = TAP_service.run_async("""
SELECT objID, RAMean, DecMean, nDetections, ng, nr, ni, nz, ny, gMeanPSFMag, rMeanPSFMag, iMeanPSFMag, zMeanPSFMag, yMeanPSFMag
FROM dbo.MeanObjectView
WHERE
CONTAINS(POINT('ICRS', RAMean, DecMean),CIRCLE('ICRS',187.706,12.391,.2))=1
AND nDetections > 1
  """, uploads={'idlist':TAP_results['objID']},verbose=True)

In [43]:
idlist = [str(id) for id in TAP_results['objID']]

In [18]:
import tempfile, os
def source_id_to_xmlfile(source_idlist, sidcol='typed_id', table_id='source_idlist'):
	#need tempfile for source id list
	fh =  tempfile.mkstemp()
	os.close(fh[0]) #fh[0] is the file descriptor; fh[1] is the path

	#xml-ify the source_idlist to a file
	tbl = Table({sidcol:source_idlist})
	tbl.write(fh[1], table_id=table_id, format='votable', overwrite=True)

	return table_id, fh[1], sidcol

In [19]:
tbl_id, fh, sidcol=source_id_to_xmlfile(idlist)

In [21]:
tbl_id, sidcol, fh

('source_idlist',
 'typed_id',
 'C:\\Users\\Kevin\\AppData\\Local\\Temp\\tmpxe0zm9yk')

In [39]:
','.join(idlist)

'122861877059169881,122861877056308967,122871877063310741,122861877058698594,122861877050678994,122871877062551436,122861877044629638,122861877056688054,122871877075050430,122861877070368181'

In [44]:
job = TAP_service.run_sync("""
SELECT objID, RAMean, DecMean, nDetections, ng, nr, ni, nz, ny, gMeanPSFMag, rMeanPSFMag, iMeanPSFMag, zMeanPSFMag, yMeanPSFMag
FROM 
dbo.MeanObjectView mov where mov.objID in ("""+','.join(idlist)+')')



In [47]:
job.to_table().to_pandas()

,objID,RAMean,DecMean,nDetections,ng,nr,ni,nz,ny,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
0,122651877531694518,187.753200,12.211626,2,0,0,2,0,0,-999.000000,-999.000000,21.880501,-999.000000,-999.0000
1,122671878002055757,187.800207,12.229504,2,0,1,1,0,0,-999.000000,21.465599,22.488199,-999.000000,-999.0000
2,122681877691724695,187.769155,12.236792,5,0,0,2,3,0,-999.000000,-999.000000,21.818800,21.263500,-999.0000
3,122681877697084332,187.769717,12.236576,3,0,2,1,0,0,-999.000000,22.032400,21.982201,-999.000000,-999.0000
4,122681877698979485,187.769881,12.240821,7,1,2,3,1,0,22.232599,21.631300,21.742001,21.186399,-999.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6922,123101877393523995,187.739383,12.586275,40,0,10,12,11,7,-999.000000,21.383600,20.734100,20.409599,19.9879
6923,123101877408345850,187.740879,12.587801,24,0,8,7,8,1,-999.000000,21.455400,21.234100,20.904400,19.4142
6924,123101877410812965,187.741057,12.585469,2,1,1,0,0,0,22.507999,21.353800,-999.000000,-999.000000,-999.0000
6925,123101877493291995,187.749321,12.584553,2,0,0,2,0,0,-999.000000,-999.000000,20.849001,-999.000000,-999.0000


In [46]:
len(idlist)

6927